In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load traning data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [3]:
df_train = pd.read_csv('./data/train.csv')
df_test =  pd.read_csv('./data/test.csv')
total = df_train.isnull().sum().sort_values(ascending=False)
print(total.head(10))

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
dtype: int64


In [4]:
# Separate numerical and categorical columns
def getNumericaldata(df,excludefield):
    quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
    quantitative.remove(excludefield)
    return quantitative

def getCategoricaldata(df):
    qualitative = [f for f in df.columns if df.dtypes[f] == 'object']
    return qualitative

numcols = getNumericaldata(df_train,'SalePrice')
catcols = getCategoricaldata(df_train)

print(numcols)
print(catcols)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'Gar

In [5]:
def datapreprocessing(df):
    df.loc[df.MasVnrType.isnull(), 'MasVnrType'] = 'None' # no good
    df.loc[df.MasVnrType == 'None', 'MasVnrArea'] = 0
    df.loc[df.LotFrontage.isnull(), 'LotFrontage'] = df.LotFrontage.median()
    df.loc[df.LotArea.isnull(), 'MasVnrType'] = 0
    df.loc[df.BsmtQual.isnull(), 'BsmtQual'] = 'NoBsmt'
    df.loc[df.BsmtCond.isnull(), 'BsmtCond'] = 'NoBsmt'
    df.loc[df.BsmtExposure.isnull(), 'BsmtExposure'] = 'NoBsmt'
    df.loc[df.BsmtFinType1.isnull(), 'BsmtFinType1'] = 'NoBsmt'
    df.loc[df.BsmtFinType2.isnull(), 'BsmtFinType2'] = 'NoBsmt'
    df.loc[df.BsmtFinType1=='NoBsmt', 'BsmtFinSF1'] = 0
    df.loc[df.BsmtFinType2=='NoBsmt', 'BsmtFinSF2'] = 0
    df.loc[df.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = df.BsmtFinSF1.median()
    df.loc[df.BsmtQual=='NoBsmt', 'BsmtUnfSF'] = 0
    df.loc[df.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = df.BsmtUnfSF.median()
    df.loc[df.BsmtQual=='NoBsmt', 'TotalBsmtSF'] = 0
    df.loc[df.FireplaceQu.isnull(), 'FireplaceQu'] = 'NoFireplace'
    df.loc[df.GarageType.isnull(), 'GarageType'] = 'NoGarage'
    df.loc[df.GarageFinish.isnull(), 'GarageFinish'] = 'NoGarage'
    df.loc[df.GarageQual.isnull(), 'GarageQual'] = 'NoGarage'
    df.loc[df.GarageCond.isnull(), 'GarageCond'] = 'NoGarage'
    df.loc[df.BsmtFullBath.isnull(), 'BsmtFullBath'] = 0
    df.loc[df.BsmtHalfBath.isnull(), 'BsmtHalfBath'] = 0
    df.loc[df.KitchenQual.isnull(), 'KitchenQual'] = 'TA'
    df.loc[df.MSZoning.isnull(), 'MSZoning'] = 'RL'
    df.loc[df.Utilities.isnull(), 'Utilities'] = 'AllPub'
    df.loc[df.Exterior1st.isnull(), 'Exterior1st'] = 'VinylSd'
    df.loc[df.Exterior2nd.isnull(), 'Exterior2nd'] = 'VinylSd'
    df.loc[df.Functional.isnull(), 'Functional'] = 'Typ'
    df.loc[df.SaleCondition.isnull(), 'SaleCondition'] = 'Normal'
    df.loc[df.SaleCondition.isnull(), 'SaleType'] = 'WD'
    df.loc[df['Electrical'].isnull(), 'Electrical'] = 'SBrkr'
    df.loc[df['SaleType'].isnull(), 'SaleType'] = 'NoSale'
    #GarageYrBlt
    df.loc[df.GarageYrBlt.isnull(), 'GarageYrBlt'] = df.GarageYrBlt.median()
    # only one is null and it has type Detchd
    df.loc[df['GarageArea'].isnull(), 'GarageArea'] = df.loc[df['GarageType']=='Detchd', 'GarageArea'].mean()
    df.loc[df['GarageCars'].isnull(), 'GarageCars'] = df.loc[df['GarageType']=='Detchd', 'GarageCars'].median()
    
#datapreprocessing(df_train)
#datapreprocessing(df_test)

In [6]:
def convertasCategory(df,cols):
    for col in cols:
        df[col] = df[col].astype('category')
        df[col+'_code'] = df[col].cat.codes
        df[col] = df[col +'_code']
        df.drop(labels=col+'_code', axis="columns", inplace=True)
    #return df
    
convertasCategory(df_train,catcols)
convertasCategory(df_test,catcols)

In [7]:
df_test_id = df_test['Id']
#print(df_test_id)

df_train.drop(['Id'],axis=1,inplace=True)
df_test.drop(['Id'],axis=1,inplace=True)

#df_train.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)
#df_test.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)

print(df_train.shape)
print(df_test.shape)

(1460, 80)
(1459, 79)


In [8]:
   

#print(df_train.head())
#print(df_train.dtypes)
# Handle remaining missing values for numerical features by using median as replacement
print("NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
df_train = df_train.fillna(df_train.median())
df_test = df_test.fillna(df_train.median())
print("Remaining NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
print("Remaining NAs for numerical features in test : " + str(df_test.isnull().values.sum()))

NAs for numerical features in train : 348
Remaining NAs for numerical features in train : 0
Remaining NAs for numerical features in test : 0


In [9]:
#Handle skewness(Reference - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])
df_train["SalePrice"].head()

0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
Name: SalePrice, dtype: float64

In [10]:
#Reference - https://www.kaggle.com/zoupet/neural-network-model-for-house-prices-tensorflow
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(df_train)
y_noano = clf.predict(df_train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

df_train = df_train.iloc[y_noano[y_noano['Top'] == 1].index.values]
df_train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", df_train.shape[0])

Number of Outliers: 146
Number of rows without outliers: 1314


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [11]:
from sklearn.model_selection import train_test_split


X = df_train.iloc[:, 1:-2].values  
#print(X[1]) 
print(X.shape)
y = df_train.iloc[:, -1].values
#print(y)

X_test = df_test.iloc[:, 1:-1].values  
#print(X_test[1]) 
print(X_test.shape)
y_test = df_test.iloc[:, -1].values
#print(y_test)


X_train, X_train_test, y_train, y_train_test = train_test_split(X, y, test_size=0.1, random_state=101)

(1314, 77)
(1459, 77)


In [12]:
# we are going to scale to data

y_train= y_train.reshape(-1,1)
y_train_test= y_train_test.reshape(-1,1)
y_test= y_train_test.reshape(-1,1)

from sklearn.preprocessing import StandardScaler
sc_X_train = StandardScaler()
sc_y_train = StandardScaler()

sc_X_train_test = StandardScaler()
sc_y_train_test = StandardScaler()

sc_X_test = StandardScaler()
sc_y_test = StandardScaler()


X_train = sc_X_train.fit_transform(X_train)
y_train = sc_y_train.fit_transform(y_train)

X_train_test = sc_X_train_test.fit_transform(X_train_test)
y_train_test = sc_y_train_test.fit_transform(y_train_test)

X_test = sc_X_test.fit_transform(X_test)
y_test = sc_y_test.fit_transform(y_test)

In [13]:
import xgboost as xgb
from xgboost import XGBRegressor
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 1000)
eval_set = [(X_train, y_train), (X_train_test, y_train_test)]
eval_metric = ["rmse","error"]
xg_reg = xgb.XGBRegressor(colsample_bytree=0.4, gamma=0,learning_rate=0.07, max_depth=3, min_child_weight=1.5,
                 n_estimators=10000, reg_alpha=0.75, reg_lambda=0.45,subsample=0.6, seed=42) 
#xg_reg = xgb.XGBRegressor(gamma=1, colsample_bytree = .5, learning_rate = 0.1,
#                max_depth = 3, alpha = 10, n_estimators = 1000)
#xgbmodel = xg_reg.fit(X,y,verbose=True)
xgbmodel = xg_reg.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)
print(xgbmodel.feature_importances_)
print("?????????????????????????????")
#X, y = make_regression(n_features=4, n_informative=2,random_state=0, shuffle=False)
#regr = RandomForestRegressor(max_depth=2, random_state=0)
#regr.fit(X, y)
#RandomForestRegressor(max_depth=2, random_state=0)
#print(regr.feature_importances_)
#print(regr.feature_importances_.shape)
#print(X.shape)
Xcols=df_train.columns
result = zip(Xcols,xgbmodel.feature_importances_.tolist())
df_impf = pd.DataFrame(set(result))
df_impf.columns = ['field','score']
print(df_impf.sort_values('score',ascending=False))


[22:31:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.0641	validation_0-error:-0.274954	validation_1-rmse:1.06424	validation_1-error:-0.274213
[1]	validation_0-rmse:1.0102	validation_0-error:-0.342617	validation_1-rmse:1.01097	validation_1-error:-0.320256
[2]	validation_0-rmse:0.965153	validation_0-error:-0.320681	validation_1-rmse:0.965603	validation_1-error:-0.330426
[3]	validation_0-rmse:0.919333	validation_0-error:-0.345888	validation_1-rmse:0.921325	validation_1-error:-0.371772
[4]	validation_0-rmse:0.879656	validation_0-error:-0.326124	validation_1-rmse:0.882391	validation_1-error:-0.359468
[5]	validation_0-rmse:0.840379	validation_0-error:-0.32792	validation_1-rmse:0.844003	validation_1-error:-0.353837
[6]	validation_0-rmse:0.806418	validation_0-error:-0.32884	validation_1-rmse:0.806739	validation_1-error:-0.354183
[7]	validation_0-rmse:0.771638	validation_0-error:-0.344417	validation_1-rmse:0.

[70]	validation_0-rmse:0.288083	validation_0-error:-0.38353	validation_1-rmse:0.330318	validation_1-error:-0.390838
[71]	validation_0-rmse:0.286828	validation_0-error:-0.384403	validation_1-rmse:0.328733	validation_1-error:-0.390838
[72]	validation_0-rmse:0.28579	validation_0-error:-0.384949	validation_1-rmse:0.328586	validation_1-error:-0.390838
[73]	validation_0-rmse:0.284905	validation_0-error:-0.385281	validation_1-rmse:0.328238	validation_1-error:-0.390838
[74]	validation_0-rmse:0.28359	validation_0-error:-0.385213	validation_1-rmse:0.328292	validation_1-error:-0.390838
[75]	validation_0-rmse:0.282778	validation_0-error:-0.384546	validation_1-rmse:0.327754	validation_1-error:-0.390838
[76]	validation_0-rmse:0.281402	validation_0-error:-0.384496	validation_1-rmse:0.326095	validation_1-error:-0.390838
[77]	validation_0-rmse:0.280423	validation_0-error:-0.384546	validation_1-rmse:0.326203	validation_1-error:-0.390838
[78]	validation_0-rmse:0.279313	validation_0-error:-0.385303	valida

[140]	validation_0-rmse:0.240812	validation_0-error:-0.390095	validation_1-rmse:0.31456	validation_1-error:-0.391661
[141]	validation_0-rmse:0.240386	validation_0-error:-0.39011	validation_1-rmse:0.314632	validation_1-error:-0.391661
[142]	validation_0-rmse:0.239966	validation_0-error:-0.390659	validation_1-rmse:0.314594	validation_1-error:-0.391661
[143]	validation_0-rmse:0.239683	validation_0-error:-0.39011	validation_1-rmse:0.314379	validation_1-error:-0.391661
[144]	validation_0-rmse:0.239503	validation_0-error:-0.390466	validation_1-rmse:0.314062	validation_1-error:-0.391661
[145]	validation_0-rmse:0.239177	validation_0-error:-0.390841	validation_1-rmse:0.313918	validation_1-error:-0.391661
[146]	validation_0-rmse:0.238539	validation_0-error:-0.390841	validation_1-rmse:0.313901	validation_1-error:-0.391661
[147]	validation_0-rmse:0.238006	validation_0-error:-0.39208	validation_1-rmse:0.314259	validation_1-error:-0.391661
[148]	validation_0-rmse:0.237534	validation_0-error:-0.39231

[210]	validation_0-rmse:0.21792	validation_0-error:-0.39279	validation_1-rmse:0.313231	validation_1-error:-0.391661
[211]	validation_0-rmse:0.217577	validation_0-error:-0.392743	validation_1-rmse:0.313669	validation_1-error:-0.391661
[212]	validation_0-rmse:0.217303	validation_0-error:-0.393134	validation_1-rmse:0.313566	validation_1-error:-0.391661
[213]	validation_0-rmse:0.217085	validation_0-error:-0.393134	validation_1-rmse:0.313468	validation_1-error:-0.391661
[214]	validation_0-rmse:0.216698	validation_0-error:-0.393647	validation_1-rmse:0.313465	validation_1-error:-0.391661
[215]	validation_0-rmse:0.21655	validation_0-error:-0.393647	validation_1-rmse:0.313222	validation_1-error:-0.391661
[216]	validation_0-rmse:0.216353	validation_0-error:-0.393316	validation_1-rmse:0.313203	validation_1-error:-0.391661
[217]	validation_0-rmse:0.215999	validation_0-error:-0.392978	validation_1-rmse:0.313267	validation_1-error:-0.391661
[218]	validation_0-rmse:0.215785	validation_0-error:-0.3930

[280]	validation_0-rmse:0.199915	validation_0-error:-0.393641	validation_1-rmse:0.314349	validation_1-error:-0.393827
[281]	validation_0-rmse:0.19973	validation_0-error:-0.393618	validation_1-rmse:0.314361	validation_1-error:-0.393827
[282]	validation_0-rmse:0.199438	validation_0-error:-0.394028	validation_1-rmse:0.314719	validation_1-error:-0.393827
[283]	validation_0-rmse:0.199234	validation_0-error:-0.394028	validation_1-rmse:0.314913	validation_1-error:-0.393827
[284]	validation_0-rmse:0.199012	validation_0-error:-0.394028	validation_1-rmse:0.315025	validation_1-error:-0.393827
[285]	validation_0-rmse:0.198788	validation_0-error:-0.394617	validation_1-rmse:0.314906	validation_1-error:-0.396112
[286]	validation_0-rmse:0.198543	validation_0-error:-0.394787	validation_1-rmse:0.315069	validation_1-error:-0.396112
[287]	validation_0-rmse:0.198199	validation_0-error:-0.394546	validation_1-rmse:0.315603	validation_1-error:-0.393923
[288]	validation_0-rmse:0.197948	validation_0-error:-0.39

...

[9701]	validation_0-rmse:0.033222	validation_0-error:-0.405845	validation_1-rmse:0.318439	validation_1-error:-0.385136
[9702]	validation_0-rmse:0.033219	validation_0-error:-0.405845	validation_1-rmse:0.318442	validation_1-error:-0.385136
[9703]	validation_0-rmse:0.033219	validation_0-error:-0.405845	validation_1-rmse:0.318442	validation_1-error:-0.385136
[9704]	validation_0-rmse:0.033219	validation_0-error:-0.405845	validation_1-rmse:0.318442	validation_1-error:-0.385136
[9705]	validation_0-rmse:0.033219	validation_0-error:-0.405845	validation_1-rmse:0.31845	validation_1-error:-0.385136
[9706]	validation_0-rmse:0.033216	validation_0-error:-0.405845	validation_1-rmse:0.318445	validation_1-error:-0.385136
[9707]	validation_0-rmse:0.033216	validation_0-error:-0.405845	validation_1-rmse:0.318446	validation_1-error:-0.385136
[9708]	validation_0-rmse:0.033213	validation_0-error:-0.405845	validation_1-rmse:0.318445	validation_1-error:-0.385136
[9709]	validation_0-rmse:0.033213	validation_0-er

[9770]	validation_0-rmse:0.033129	validation_0-error:-0.405846	validation_1-rmse:0.318531	validation_1-error:-0.385136
[9771]	validation_0-rmse:0.033128	validation_0-error:-0.405846	validation_1-rmse:0.318534	validation_1-error:-0.385136
[9772]	validation_0-rmse:0.033126	validation_0-error:-0.405846	validation_1-rmse:0.318554	validation_1-error:-0.385136
[9773]	validation_0-rmse:0.033122	validation_0-error:-0.405846	validation_1-rmse:0.318564	validation_1-error:-0.385136
[9774]	validation_0-rmse:0.03312	validation_0-error:-0.405846	validation_1-rmse:0.318579	validation_1-error:-0.385136
[9775]	validation_0-rmse:0.03312	validation_0-error:-0.405846	validation_1-rmse:0.318579	validation_1-error:-0.385136
[9776]	validation_0-rmse:0.033117	validation_0-error:-0.405846	validation_1-rmse:0.318574	validation_1-error:-0.385136
[9777]	validation_0-rmse:0.033115	validation_0-error:-0.405846	validation_1-rmse:0.318574	validation_1-error:-0.385136
[9778]	validation_0-rmse:0.033115	validation_0-err

[9839]	validation_0-rmse:0.03304	validation_0-error:-0.405846	validation_1-rmse:0.318593	validation_1-error:-0.385136
[9840]	validation_0-rmse:0.033035	validation_0-error:-0.405846	validation_1-rmse:0.318611	validation_1-error:-0.385136
[9841]	validation_0-rmse:0.033035	validation_0-error:-0.405846	validation_1-rmse:0.318611	validation_1-error:-0.385136
[9842]	validation_0-rmse:0.033034	validation_0-error:-0.405846	validation_1-rmse:0.318607	validation_1-error:-0.385136
[9843]	validation_0-rmse:0.033032	validation_0-error:-0.405846	validation_1-rmse:0.318607	validation_1-error:-0.385136
[9844]	validation_0-rmse:0.033031	validation_0-error:-0.405846	validation_1-rmse:0.318603	validation_1-error:-0.385136
[9845]	validation_0-rmse:0.033028	validation_0-error:-0.405846	validation_1-rmse:0.318599	validation_1-error:-0.385136
[9846]	validation_0-rmse:0.033027	validation_0-error:-0.405846	validation_1-rmse:0.318597	validation_1-error:-0.385136
[9847]	validation_0-rmse:0.033026	validation_0-er

[9909]	validation_0-rmse:0.032945	validation_0-error:-0.405846	validation_1-rmse:0.318518	validation_1-error:-0.385136
[9910]	validation_0-rmse:0.032945	validation_0-error:-0.405846	validation_1-rmse:0.318518	validation_1-error:-0.385136
[9911]	validation_0-rmse:0.032943	validation_0-error:-0.405846	validation_1-rmse:0.318518	validation_1-error:-0.385136
[9912]	validation_0-rmse:0.032939	validation_0-error:-0.405846	validation_1-rmse:0.318512	validation_1-error:-0.385136
[9913]	validation_0-rmse:0.032939	validation_0-error:-0.405846	validation_1-rmse:0.318512	validation_1-error:-0.385136
[9914]	validation_0-rmse:0.032938	validation_0-error:-0.405846	validation_1-rmse:0.318506	validation_1-error:-0.385136
[9915]	validation_0-rmse:0.032933	validation_0-error:-0.405846	validation_1-rmse:0.318522	validation_1-error:-0.385136
[9916]	validation_0-rmse:0.032933	validation_0-error:-0.405846	validation_1-rmse:0.318522	validation_1-error:-0.385136
[9917]	validation_0-rmse:0.032929	validation_0-e

[9978]	validation_0-rmse:0.032856	validation_0-error:-0.405846	validation_1-rmse:0.318502	validation_1-error:-0.385136
[9979]	validation_0-rmse:0.032854	validation_0-error:-0.405846	validation_1-rmse:0.318501	validation_1-error:-0.385136
[9980]	validation_0-rmse:0.032853	validation_0-error:-0.405846	validation_1-rmse:0.318509	validation_1-error:-0.385136
[9981]	validation_0-rmse:0.032851	validation_0-error:-0.405846	validation_1-rmse:0.318505	validation_1-error:-0.385136
[9982]	validation_0-rmse:0.03285	validation_0-error:-0.405846	validation_1-rmse:0.318489	validation_1-error:-0.385136
[9983]	validation_0-rmse:0.032848	validation_0-error:-0.405846	validation_1-rmse:0.318476	validation_1-error:-0.385136
[9984]	validation_0-rmse:0.032846	validation_0-error:-0.405846	validation_1-rmse:0.31847	validation_1-error:-0.385136
[9985]	validation_0-rmse:0.032845	validation_0-error:-0.405846	validation_1-rmse:0.318472	validation_1-error:-0.385136
[9986]	validation_0-rmse:0.032844	validation_0-err

In [14]:
predictions = xgbmodel.predict(X_test)
print(predictions)
print(predictions.shape)
predictions = np.expm1(sc_y_test.inverse_transform(predictions))
df_predictions = pd.DataFrame(predictions, columns=['SalePrice'])
#df_predictions.head()

#submission data
df_sub1 = pd.concat([df_test_id,df_predictions],axis=1)
print(df_sub1.head())
df_sub1.to_csv('./data/submission1.csv',index=False)

[-1.1030146  -0.18220001  0.15190804 ... -0.07315904 -1.1026258
  0.72703624]
(1459,)
     Id      SalePrice
0  1461  106269.281250
1  1462  151003.953125
2  1463  171534.015625
3  1464  200099.765625
4  1465  174239.781250


In [15]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import numpy as np

svr = SVR(kernel = 'rbf',gamma='auto')
svr.fit(X_train, y_train)

svr_pred = np.expm1(sc_y_test.inverse_transform(svr.predict(X_train_test)))
svr_pred= svr_pred.reshape(-1,1)

print('MAE:', metrics.mean_absolute_error(y_train_test, svr_pred))
print('MSE:', metrics.mean_squared_error(y_train_test, svr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train_test, svr_pred)))

MAE: 172166.00572865768
MSE: 33658820470.460835
RMSE: 183463.40362715622


In [16]:
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
params1={'n_estimators':3000, 'learning_rate':0.05, 'max_depth':4, 'max_features':'sqrt',
                                   'min_samples_leaf':15, 'min_samples_split':2, 
                                   'loss':'huber', 'random_state':5}
clf = ensemble.GradientBoostingRegressor(**params1)

clf.fit(X_train, y_train)

clf_pred=np.expm1(sc_y_train_test.inverse_transform(clf.predict(X_train_test)))
clf_pred= clf_pred.reshape(-1,1)

print('MAE:', metrics.mean_absolute_error(y_train_test, clf_pred))
print('MSE:', metrics.mean_squared_error(y_train_test, clf_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train_test, clf_pred)))



MAE: 172640.99918490875
MSE: 34477413621.11142
RMSE: 185680.945767495


In [17]:
#clf_pred =np.expm1(clf_pred)
#print(clf_pred)
df_clf_pred = pd.DataFrame(clf_pred, columns=['SalePrice'])
df_clf_pred.head()

,SalePrice
0,92847.409701
1,284324.060101
2,406359.697365
3,132572.977463
4,147808.255196


In [18]:
#try gbr on test data
clf_pred=clf.predict(X_test)
clf_pred= clf_pred.reshape(-1,1)
clf_pred = np.expm1(sc_y_test.inverse_transform(clf_pred))
#print(clf_pred)
df_clf_pred = pd.DataFrame(clf_pred, columns=['SalePrice'])
df_clf_pred.head()

,SalePrice
0,111915.041431
1,162512.257853
2,178264.727523
3,194752.031616
4,175700.505364


In [19]:
#submission data
df_sub = pd.concat([df_test_id,df_clf_pred],axis=1)
print(df_sub.head())

     Id      SalePrice
0  1461  111915.041431
1  1462  162512.257853
2  1463  178264.727523
3  1464  194752.031616
4  1465  175700.505364


In [20]:
df_sub.to_csv('./data/submission.csv',index=False)

Change Log
++++++++++++++++++++++++++++++++++=

Feartue Engineering
Replace missing values by median

Model
SVR

Score
MAE: 50138.725050989204
    
MSE: 5299712145.626122
    
RMSE: 72799.12187400424

+++++++++++++++++++++++++++++++++
Params1
MAE: 0.21022910696011998
MSE: 0.10793411607512507
RMSE: 0.3285332800115158
    
Params
MAE: 0.21493207760478975
MSE: 0.11624055545701495
RMSE: 0.3409406919935122



In [21]:
#Try out encoding as done here - https://www.kaggle.com/yingbao/feature-engineering-for-house-price-prediction
# Change the categorical data to numerical data. Avoid using get_dummies at this stage, because it will generate too many variables.
# function to add an encoded column for each categorical variable
"""
def encode(frame, feature):
    ordering = pd.DataFrame()
    # add a column of 'val' to dataframe. values in 'val' is the values in a specific categorical column
    ordering['val'] = frame[feature].unique()
    print(ordering['val'])
    # add index
    ordering.index = ordering.val
    # spmean is based on the average SalePrice for each categorical value (group by each categorical value)
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    print("*************************************")
    print(ordering['spmean'])
    # sort according to the spmean and rank it
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    # convert it to a dictionary
    ordering = ordering['ordering'].to_dict()
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print(ordering)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    # the a new column _E to the orginal df_train. 
    for cat, o in ordering.items():
        print(cat)
        print(o)
        frame.loc[frame[feature]==cat, feature + '_E'] = o
        
# the new columns provide quick influence estimations of categorical variable on SalePrice
# here, we just want to get a rough idea, so consider other variables as fixed
qual_encoded = []
qualitative=catcols
for q in qualitative:
    encode(df_train, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)
"""

'\ndef encode(frame, feature):\n    ordering = pd.DataFrame()\n    # add a column of \'val\' to dataframe. values in \'val\' is the values in a specific categorical column\n    ordering[\'val\'] = frame[feature].unique()\n    print(ordering[\'val\'])\n    # add index\n    ordering.index = ordering.val\n    # spmean is based on the average SalePrice for each categorical value (group by each categorical value)\n    ordering[\'spmean\'] = frame[[feature, \'SalePrice\']].groupby(feature).mean()[\'SalePrice\']\n    print("*************************************")\n    print(ordering[\'spmean\'])\n    # sort according to the spmean and rank it\n    ordering = ordering.sort_values(\'spmean\')\n    ordering[\'ordering\'] = range(1, ordering.shape[0]+1)\n    # convert it to a dictionary\n    ordering = ordering[\'ordering\'].to_dict()\n    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")\n    print(ordering)\n    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")\n    # the a new column _E to the orgina

In [22]:
### References
# Remove Outliers - https://www.kaggle.com/zoupet/neural-network-model-for-house-prices-tensorflow 
# GridSearchCV - https://medium.com/datadriveninvestor/an-introduction-to-grid-search-ff57adcc0998
# Skewness - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
#
#